In [1]:
import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths


In [2]:
# imports for training
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
# import dataset, network to train and metric to optimize
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss

In [3]:
%env CLEARML_WEB_HOST=http://148.253.62.107:8080
%env CLEARML_API_HOST=http://148.253.62.107:8008
%env CLEARML_FILES_HOST=http://148.253.62.107:8081
%env CLEARML_API_ACCESS_KEY=OL9OG6XAEEURZXY317LA
%env CLEARML_API_SECRET_KEY=O5xABmDwofVJoXRQcAODx3IQWFksGGtUf2S6JCBSDRCm4hsfqx

env: CLEARML_WEB_HOST=http://148.253.62.107:8080
env: CLEARML_API_HOST=http://148.253.62.107:8008
env: CLEARML_FILES_HOST=http://148.253.62.107:8081
env: CLEARML_API_ACCESS_KEY=OL9OG6XAEEURZXY317LA
env: CLEARML_API_SECRET_KEY=O5xABmDwofVJoXRQcAODx3IQWFksGGtUf2S6JCBSDRCm4hsfqx


In [4]:
from clearml import Task
task = Task.init(project_name='TemporalFusionTransformer', task_name='Experiment Wind')

ClearML Task: created new task id=9203b4ec0d1c40d0ac1b43bcf9fc7fcd
ClearML results page: http://148.253.62.107:8080/projects/43606053e6db48478ba1120f9ba5322b/experiments/9203b4ec0d1c40d0ac1b43bcf9fc7fcd/output/log


In [5]:
df = pd.read_csv('/home/kpavel/PycharmProjects/energy/data/wind/data_preprocessed.csv', sep=',')

In [6]:
df['time_idx_column'] = df.index

In [7]:
df['id'] = 0

In [8]:
df.shape

(6514, 9)

In [9]:
df = df.dropna(subset=['dt'], axis=0)

In [10]:
df = df.dropna(axis=0)


2022-12-30 16:08:00,854 - clearml.Task - INFO - Storing jupyter notebook directly as code


In [11]:
df[120 * 24:].shape

(3634, 9)

In [12]:
#df['dt'] = df['dt'].astype(np.datetime64).astype(int)

In [13]:

# load data: this is pandas dataframe with at least a column for
# * the target (what you want to predict)
# * the timeseries ID (which should be a unique string to identify each timeseries)
# * the time of the observation (which should be a monotonically increasing integer)
# data = df

# define the dataset, i.e. add metadata to pandas dataframe for the model to understand it
max_prediction_length = 14
max_encoder_length = 2 * max_prediction_length
# training_cutoff = "2022-07-25 04:00:00"  # day for cutoff

In [23]:
from pytorch_forecasting import NaNLabelEncoder
training_cutoff = df["time_idx_column"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx_column <= training_cutoff],
    time_idx= 'time_idx_column',  # column name of time of observation
    target= 'AP',  # column name of target to predict
    group_ids=[ 'id' ],  # column name(s) for timeseries IDs
    # allow_missing_timesteps=True,
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,  # how much history to use
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,  # how far to predict into future
   # categorical_encoders={data.columns[3]: NaNLabelEncoder(add_nan=True)}
    add_relative_time_idx=True,
)

In [24]:

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, df,
                                            predict=False,
                                            stop_randomization=True
                                            )

# create dataloaders for model
batch_size = 64  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [22]:
# iss = []
# for i, el in enumerate(train_dataloader):
#     iss.append(i)
#
# print(len(iss))
# iss = []
# for i, el in enumerate(val_dataloader):
#     iss.append(i)
#
# print(len(iss))

102


In [17]:

# create PyTorch Lighning Trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
lr_logger = LearningRateMonitor()
trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,  # run on CPU, if on multiple GPUs, use accelerator="ddp"
    gradient_clip_val=0.1,
    limit_train_batches=30,  # 30 batches per epoch
    callbacks=[lr_logger, early_stop_callback],
    logger=TensorBoardLogger("lightning_logs")
)

/home/kpavel/anaconda3/envs/energy/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning:

Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=1,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=3,  # 7 quantiles by default
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
/home/kpavel/anaconda3/envs/energy/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning:

Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 15.8k


In [19]:
res = trainer.tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

Missing logger folder: /home/kpavel/PycharmProjects/energy/experiments/wind/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Restoring states from the checkpoint path at /home/kpavel/PycharmProjects/energy/experiments/wind/.lr_find_a211548f-0bed-429c-b20b-0fa051aa60aa.ckpt


KeyError: 'radam_buffer'

In [29]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=5,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16 * 2,
    attention_head_size=4,
    dropout=0.2,
    hidden_continuous_size=8 * 1,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=3,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

/home/kpavel/anaconda3/envs/energy/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning:

Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 55.7k


In [30]:

# fit the model on the data - redefine the model with the correct learning rate if necessary
trainer.fit(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 32    
4  | static_variable_selection          | VariableSelectionNetwork        | 896   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 896   
6  | decoder_variable_selection         | VariableSelectionNetwork        | 896   
7  | static_context_variable_selection  | GatedResidualNetwork            | 4.3 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 4.3 K 
9  | static_context_initial_cell_lstm 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

2022-12-30 16:14:02,853 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:14:02,855 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:14:03,842 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:14:03,844 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:14:04,807 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:14:04,809 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:14:06,937 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:14:06,939 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:14:08,632 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:14:08,634 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newa

...

2022-12-30 16:29:19,247 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:19,250 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:20,695 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:20,697 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:21,917 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:21,919 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:23,545 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:23,547 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:25,702 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:25,704 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newa

Validation: 0it [00:00, ?it/s]

2022-12-30 16:29:37,881 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:37,883 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:39,219 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:39,220 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:40,483 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:40,484 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:41,934 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:41,936 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:43,739 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:43,740 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newa

Validation: 0it [00:00, ?it/s]

2022-12-30 16:29:55,147 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:55,150 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:56,385 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:56,387 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:57,801 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:57,803 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:29:59,257 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:29:59,259 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newaxis)
2022-12-30 16:30:00,892 - clearml - WARNING - Failed decoding debug image [640, 480, 3]
2022-12-30 16:30:00,894 - clearml - WARNING - Error: cannot reshape array of size 1 into shape (480,640,newa

In [31]:
trainer.save_checkpoint("TFT_model_wind")

In [32]:
import torch
torch.save(tft.state_dict(), "TFT_model_wind_weights.pt")

In [33]:
task.upload_artifact('trainer_checkpoint', artifact_object='TFT_model_wind')
task.upload_artifact('model_weights', artifact_object='TFT_model_wind_weights.pt')

True

In [34]:
task.close()

# Darts

In [5]:
import pandas as pd
from darts import TimeSeries

# Read a pandas DataFrame
# df = pd.read_csv("AirPassengers.csv", delimiter=",")

In [6]:
cols = [c for c in df.columns if c not in ['dt','gtpp', 'id','load_time', 'predict']]

In [7]:
# cols

In [8]:
df = df.drop_duplicates(['dt'], keep='last')

In [9]:
df = df.reset_index(drop=True)

In [10]:
df

,id,gtpp,dt,load_time,predict,10_metre_V_wind_component,Snow_density,Snowfall,Visibility,Surface_pressure,...,Low_cloud_cover,Snow_depth,High_cloud_cover,Evaporation,Wind_Direction,2_metre_dewpoint_temperature,Total_precipitation,2_metre_relative_humidity,Clear_sky_direct_solar_radiation_at_surface,Snow_height
0,2835,PROZ0001,2021-02-09 00:00:00,2021-02-09 15:39:45,0,-0.81,192.1333,0.0,1.115134e+04,94916.3,...,0.84,0.0983,0.00,0.000000,280.0,-31.4,0.0000,65.0,0.00,511.4249
1,2836,PROZ0001,2021-02-09 01:00:00,2021-02-09 15:39:45,60,-0.78,192.1216,0.0,1.038648e+04,95000.2,...,0.87,0.0983,0.00,-0.000004,280.0,-31.7,0.0000,65.0,0.00,511.4542
2,2837,PROZ0001,2021-02-09 02:00:00,2021-02-09 15:39:45,120,-0.55,192.1035,0.0,8.074824e+03,95084.4,...,0.94,0.0984,0.00,-0.000008,277.0,-31.7,0.0000,67.0,0.00,511.8842
3,2838,PROZ0001,2021-02-09 03:00:00,2021-02-09 15:39:45,180,-0.29,192.0900,0.0,7.603944e+03,95160.7,...,0.96,0.0984,0.00,-0.000010,274.0,-31.9,0.0000,69.0,0.00,511.9207
4,2839,PROZ0001,2021-02-09 04:00:00,2021-02-09 15:39:45,240,-0.02,192.0899,0.0,7.627873e+03,95273.9,...,0.98,0.0984,0.00,-0.000013,270.0,-32.0,0.0000,70.0,1866.71,511.9210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11653,6523892,PROZ0001,2022-08-01 06:00:00,2022-08-01 10:51:03,360,-1.58,0.0000,0.0,2.412450e+07,95059.7,...,38.08,0.0000,48.84,0.000000,22.4,12.3,0.0000,56.0,3342182.40,0.0000
11654,6523893,PROZ0001,2022-08-01 07:00:00,2022-08-01 10:51:03,420,-2.05,0.0000,0.0,2.228528e+07,95057.5,...,1.76,0.0000,98.31,0.000000,23.6,11.9,0.0625,51.0,7911388.80,0.0000
11655,6523894,PROZ0001,2022-08-01 08:00:00,2022-08-01 10:51:03,480,-2.66,0.0000,0.0,2.413507e+07,95053.5,...,39.60,0.0000,53.74,0.000000,24.7,10.6,0.0000,44.0,10427572.80,0.0000
11656,6523895,PROZ0001,2022-08-01 09:00:00,2022-08-01 10:51:03,540,-2.79,0.0000,0.0,2.413519e+07,95040.7,...,72.03,0.0000,42.14,0.000000,24.8,10.2,0.0000,42.0,12576268.80,0.0000


In [11]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import missingno as mno
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)
from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import mape, rmse
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression
pd.set_option("display.precision",2)
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = '{:,.2f}'.format

In [12]:
LOAD = False         # True = load previously saved model from disk?  False = (re)train the model
SAVE = "/exp1/_TFT_model_02.pth.tar"   # file name to save the model under

EPOCHS = 200
INLEN = 32          # input size
HIDDEN = 64         # hidden layers
LSTMLAYERS = 2      # recurrent layers
ATTH = 4            # attention heads
BATCH = 32          # batch size
LEARN = 1e-3        # learning rate
DROPOUT = 0.1       # dropout rate
VALWAIT = 1         # epochs to wait before evaluating the loss on the test/validation set
N_FC = 1            # output size

RAND = 42           # random seed
N_SAMPLES = 100     # number of times a prediction is sampled from a probabilistic model
N_JOBS = 3          # parallel processors to use;  -1 = all processors

# default quantiles for QuantileRegression
QUANTILES = [0.01, 0.1, 0.2, 0.5, 0.8, 0.9, 0.99]

SPLIT = 0.9         # train/test %

FIGSIZE = (9, 6)

qL1, qL2 = 0.01, 0.10        # percentiles of predictions: lower bounds
qU1, qU2 = 1-qL1, 1-qL2,     # upper bounds derived from lower bounds
label_q1 = f'{int(qU1 * 100)} / {int(qL1 * 100)} percentile band'
label_q2 = f'{int(qU2 * 100)} / {int(qL2 * 100)} percentile band'

mpath = os.path.abspath(os.getcwd()) + SAVE

In [91]:
mpath

'/home/kpavel/PycharmProjects/ITMO_Hack_2022/experiments/exp1/_TFT_model_02.pth.tar'

In [92]:
# load
# df0 = pd.read_csv("energy_dataset.csv", header=0, parse_dates=["time"])
# dfw0 = pd.read_csv("weather_features.csv", header=0, parse_dates=["dt_iso"])

In [13]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, time_col="dt", value_cols=cols, fill_missing_dates=True, freq='H')
# Set aside the last 36 months as a validation series
# train, val = series[:-36], series[-36:]

In [14]:
# create time series object for target variable
ts_P = TimeSeries.from_series(df["predict"])

# check attributes of the time series
print("components:", ts_P.components)
print("duration:",ts_P.duration)
print("frequency:",ts_P.freq)
print("frequency:",ts_P.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_P.has_datetime_index)
print("deterministic:",ts_P.is_deterministic)
print("univariate:",ts_P.is_univariate)

components: Index(['predict'], dtype='object', name='component')
duration: 11657
frequency: 1
frequency: None
has date time index? (or else, it must have an integer index): False
deterministic: True
univariate: True


In [15]:
# create time series object for the feature columns
df_covF = df.loc[:, cols]
ts_covF = TimeSeries.from_dataframe(df_covF)

# check attributes of the time series
print("components (columns) of feature time series:", ts_covF.components)
print("duration:",ts_covF.duration)
print("frequency:",ts_covF.freq)
print("frequency:",ts_covF.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_covF.has_datetime_index)
print("deterministic:",ts_covF.is_deterministic)
print("univariate:",ts_covF.is_univariate)

components (columns) of feature time series: Index(['10_metre_V_wind_component', 'Snow_density', 'Snowfall', 'Visibility',
       'Surface_pressure', 'Convective_precipitation', 'Visual_cloud_cover',
       'Total_cloud_cover', 'Precipitation_type',
       'Instantaneous_10_metre_wind_gust', 'Medium_cloud_cover',
       'Total_precipitation_rate', 'Convective_available_potential_energy',
       '10_metre_U_wind_component', 'Skin_temperature', '2_metre_temperature',
       'Surface_solar_radiation_downwards', 'Wind_speed', 'Low_cloud_cover',
       'Snow_depth', 'High_cloud_cover', 'Evaporation', 'Wind_Direction',
       '2_metre_dewpoint_temperature', 'Total_precipitation',
       '2_metre_relative_humidity',
       'Clear_sky_direct_solar_radiation_at_surface', 'Snow_height'],
      dtype='object', name='component')
duration: 11657
frequency: 1
frequency: None
has date time index? (or else, it must have an integer index): False
deterministic: True
univariate: False


In [16]:
# train/test split and scaling of target variable
ts_train, ts_test = ts_P.split_after(SPLIT)
print("training start:", ts_train.start_time())
print("training end:", ts_train.end_time())
print("training duration:",ts_train.duration)
print("test start:", ts_test.start_time())
print("test end:", ts_test.end_time())
print("test duration:", ts_test.duration)


scalerP = Scaler()
scalerP.fit_transform(ts_train)
ts_ttrain = scalerP.transform(ts_train)
ts_ttest = scalerP.transform(ts_test)
ts_t = scalerP.transform(ts_P)

# make sure data are of type float
ts_t = ts_t.astype(np.float32)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)

print("first and last row of scaled price time series:")
pd.options.display.float_format = '{:,.2f}'.format
ts_t.pd_dataframe().iloc[[0,-1]]

training start: 0
training end: 10491
training duration: 10491
test start: 10492
test end: 11657
test duration: 1165
first and last row of scaled price time series:


component,predict
time,
0,0.00
11657,0.43


In [17]:
# train/test split and scaling of feature covariates
covF_train, covF_test = ts_covF.split_after(SPLIT)

scalerF = Scaler()
scalerF.fit_transform(covF_train)
covF_ttrain = scalerF.transform(covF_train)
covF_ttest = scalerF.transform(covF_test)
covF_t = scalerF.transform(ts_covF)

# make sure data are of type float
covF_ttrain = covF_ttrain.astype(np.float32)
covF_ttest = covF_ttest.astype(np.float32)

pd.options.display.float_format = '{:.2f}'.format
print("first and last row of scaled feature covariates:")
covF_t.pd_dataframe().iloc[[0,-1]]

first and last row of scaled feature covariates:


component,10_metre_V_wind_component,Snow_density,Snowfall,Visibility,Surface_pressure,Convective_precipitation,Visual_cloud_cover,Total_cloud_cover,Precipitation_type,Instantaneous_10_metre_wind_gust,...,Low_cloud_cover,Snow_depth,High_cloud_cover,Evaporation,Wind_Direction,2_metre_dewpoint_temperature,Total_precipitation,2_metre_relative_humidity,Clear_sky_direct_solar_radiation_at_surface,Snow_height
time,,,,,,,,,,,,,,,,,,,,,
0,0.44,0.50,0.00,0.00,0.48,0.00,0.01,0.01,0.07,0.48,...,0.01,0.00,0.00,0.12,0.79,0.03,0.00,0.61,0.00,0.00
11657,0.31,0.03,0.00,0.47,0.50,0.01,0.57,0.78,1.00,0.00,...,0.39,0.00,0.61,0.12,0.09,0.84,0.00,0.43,0.54,0.00


In [18]:
# feature engineering - create time covariates: hour, weekday, month, year, country-specific holidays
covT = datetime_attribute_timeseries( ts_P.time_index, attribute="hour", add_length=48 )   # 48 hours beyond end of test set to prepare for out-of-sample forecasting
covT = covT.stack(  datetime_attribute_timeseries(covT.time_index, attribute="day_of_week")  )
covT = covT.stack(  datetime_attribute_timeseries(covT.time_index, attribute="month")  )
covT = covT.stack(  datetime_attribute_timeseries(covT.time_index, attribute="year")  )

covT = covT.add_holidays(country_code="RU")
covT = covT.astype(np.float32)

# train/test split
covT_train, covT_test = covT.split_after(ts_train.end_time())


# rescale the covariates: fitting on the training set
scalerT = Scaler()
scalerT.fit(covT_train)
covT_ttrain = scalerT.transform(covT_train)
covT_ttest = scalerT.transform(covT_test)
covT_t = scalerT.transform(covT)

covT_t = covT_t.astype(np.float32)

pd.options.display.float_format = '{:.0f}'.format
print("first and last row of unscaled time covariates:")
covT.pd_dataframe().iloc[[0,-1]]

AttributeError: 'RangeIndex' object has no attribute 'freq'

In [19]:
# dfX["ysin"] = np.sin( 2 * np.pi * dfX["week"] / (365.25/7) )
# dfX["ycos"] = np.cos( 2 * np.pi * dfX["week"] / (365.25/7) )

In [20]:
# combine feature and time covariates along component dimension: axis=1
ts_cov = ts_covF.concatenate( covT.slice_intersect(ts_covF), axis=1 )                      # unscaled F+T
cov_t = covF_t.concatenate( covT_t.slice_intersect(covF_t), axis=1 )                       # scaled F+T
cov_ttrain = covF_ttrain.concatenate( covT_ttrain.slice_intersect(covF_ttrain), axis=1 )   # scaled F+T training set
cov_ttest = covF_ttest.concatenate( covT_ttest.slice_intersect(covF_ttest), axis=1 )       # scaled F+T test set

pd.options.display.float_format = '{:.2f}'.format
print("first and last row of unscaled covariates:")
ts_cov.pd_dataframe().iloc[[0,-1]]

NameError: name 'covT' is not defined

In [21]:
ts_cov = ts_covF                     # unscaled F+T
cov_t = covF_t                      # scaled F+T
cov_ttrain = covF_ttrain   # scaled F+T training set
cov_ttest = covF_ttest      # scaled F+T test set

pd.options.display.float_format = '{:.2f}'.format
print("first and last row of unscaled covariates:")
ts_cov.pd_dataframe().iloc[[0,-1]]

first and last row of unscaled covariates:


component,10_metre_V_wind_component,Snow_density,Snowfall,Visibility,Surface_pressure,Convective_precipitation,Visual_cloud_cover,Total_cloud_cover,Precipitation_type,Instantaneous_10_metre_wind_gust,...,Low_cloud_cover,Snow_depth,High_cloud_cover,Evaporation,Wind_Direction,2_metre_dewpoint_temperature,Total_precipitation,2_metre_relative_humidity,Clear_sky_direct_solar_radiation_at_surface,Snow_height
time,,,,,,,,,,,,,,,,,,,,,
0,-0.81,192.13,0.00,11151.34,94916.30,0.00,0.84,0.84,5.00,10.20,...,0.84,0.10,0.00,0.00,280.00,-31.40,0.00,65.00,0.00,511.42
11657,-2.60,0.00,0.00,24134921.98,95044.20,0.12,57.43,78.33,255.00,0.00,...,39.28,0.00,61.40,0.00,22.50,11.10,0.00,49.00,12831796.80,0.00


In [41]:

model = TFTModel(   input_chunk_length=INLEN,
                    output_chunk_length=N_FC,
                    hidden_size=HIDDEN,
                    lstm_layers=LSTMLAYERS,
                    num_attention_heads=ATTH,
                    dropout=DROPOUT,
                    batch_size=BATCH,
                    n_epochs=1,#EPOCHS,
                    nr_epochs_val_period=VALWAIT,
                    likelihood=QuantileRegression(QUANTILES),
                    optimizer_kwargs={"lr": LEARN},
                    model_name="TFT_EnergyES",
                    log_tensorboard=True,
                    random_state=RAND,
                    force_reset=True,
                    save_checkpoints=True
                )

In [42]:
cov_t = cov_t.astype('float32')

In [43]:
# training: load a saved model or (re)train
if LOAD:
    print("have loaded a previously saved model from disk:" + mpath)
    model = TFTModel.load_model(mpath)                            # load previously model from disk
else:
    model.fit(series=ts_ttrain,
              future_covariates=cov_t,
              val_series=ts_ttest,
              val_future_covariates=cov_t,
              verbose=True)
    print("have saved the model after training:", mpath)
    model.save_model('TFT_model_02.pth.tar')

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

have saved the model after training: /home/kpavel/PycharmProjects/ITMO_Hack_2022/experiments/exp1/_TFT_model_02.pth.tar


In [105]:
model.save_model('TFT_model_02.pth.tar')

In [33]:
model.load_model('TFT_model_02.pth.tar')

In [60]:
ts_tpred = model.predict(#series=ts_ttest,
                         n=len(ts_ttest),
                         num_samples=N_SAMPLES,
                         n_jobs=N_JOBS,
                         verbose=True)

Predicting: 327it [00:00, ?it/s]